In [2]:
!pip install psycopg2-binary SQLAlchemy pymysql

In [10]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd 
import pymongo 
from json import loads

host = "mongodb://localhost/"
port = 27017
conector = pymongo.MongoClient(host,port)
database = conector["sistema_mongo"]
categorias = database["categorias"]
produtos = database["produtos"]

In [4]:
host_postgres = "localhost"
user_postgres = "postgres"
password_postgres = "POSTGRES"
database_postgres = "datawarehouse"
engine_postgres = create_engine(f"postgresql://{user_postgres}:{password_postgres}@{host_postgres}/{database_postgres}")

if not database_exists(engine_postgres.url):
    create_database(engine_postgres.url)

In [26]:
def executar_query(tabela):
    df = pd.DataFrame(list(tabela.find({})))
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'id'})
    df = df.drop("_id", axis = 1)
    df["id"] = df["id"].apply(lambda x: x+1)
    return df

def criando_tabela(nome_da_tabela, df):
    with engine_postgres.connect() as conn:
        df.to_sql(nome_da_tabela, conn, index=False, if_exists='replace')
        

In [27]:
criando_tabela("categorias", executar_query(categorias))
criando_tabela("produtos", executar_query(produtos))